In [ ]:
import numpy as np, os
import pandas as pd

import optuna

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

DEBUG = False
TRAIN_MODEL = False
INFER_TEST = True
ONE_FOLD_ONLY = True
COMPUTE_LSTM_IMPORTANCE = True

train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
pressure_values = np.sort( train.pressure.unique() )
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

if DEBUG:
    train = train[:80*1000]

In [ ]:
#train.to_csv('trainWithOutliers.csv')
count = 0;
deleteList = []
for index,item in train.iterrows():
    if item['pressure'] > 63:
        #print(item['breath_id'])
        deleteList.append(item['breath_id'])
    count = count + 1
    if count % 100000 == 0:
        print(len(train.index)-count)
deleteList = list(set(deleteList))
print(deleteList)


In [ ]:
for i in deleteList:
    train = train[train['breath_id']!=i]

In [ ]:
train.head()

In [ ]:
train = train.set_index('id')
train.to_csv('trainWithoutOutliers.csv')

In [ ]:
train.head()